In [143]:
import nltk
import random
import os
from sklearn import svm
import json
from nltk.stem import  PorterStemmer
import random
import pickle
import numpy as np

In [125]:
label_map={'neg':0 ,'sli_neg':1 ,'neutral':2 ,'sli_pos':3 ,'pos':4}

In [126]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,label_map[each_file]) for sentence in data_temp]
    return dataset

In [127]:
#initialising porterstemmer
ps=PorterStemmer()

In [128]:
def to_unigram(data):
    dataset=[]
    dataset+=[([items for items in sentence.split()],polarity) for sentence,polarity in data]    
    return dataset    

In [129]:
def get_words(x):    #returns all the words in the  data set
    vocab = []
    for sentence,polarity in x:  
        vocab.extend(sentence.split())
    return vocab

In [130]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)
vocab=set(get_words(labeled_data)) #getting words 
vocab=[ps.stem(word) for word in vocab]   #stemming
with open('vocab_unigram','w') as f:
    json.dump(vocab,f)
labeled_data=to_unigram(labeled_data)    #converting to unigram

In [131]:
#bigrams
print labeled_data[:5]

[([u'made', u'lot', u'chang', u'career', u'also', u'life'], 2), ([u'work', u'inexperienc', u'worker'], 1), ([u'el', u'nice', u'problem', u'con', u'oracl', u'es', u'es', u'may', u'rest'], 4), ([u'learn', u'quit', u'bit', u'regard', u'manag', u'team'], 2), ([u'good', u'work', u'environ', u'depend', u'citi', u'work', u'well'], 4)]


In [132]:
print len(labeled_data)

201378


In [133]:
print len(vocab)
print vocab[:100]

6544
[u'yellow', u'interchang', u'four', u'payoff', u'authorit', u'scold', u'lore', u'lord', u'digit', u'pigment', u'foul', u'delv', u'politician', u'disturb', u'prize', u'wooden', u'showca', u'habea', u'charter', u'corpora', u'nigh', u'tire', u'miller', u'histor', u'second', u'sooth', u'tether', u'dialogu', u'ruthless', u'thunder', u'specialist', u'aver', u'intellectu', u'hero', u'avert', u'swag', u'herd', u'gamut', u'china', u'cult', u'deterior', u'militari', u'k', u'pastur', u'fumbl', u'diplomat', u'golden', u'brought', u'sterl', u'stern', u'unit', u'spoke', u'overshadow', u'music', u'telegraph', u'passport', u'strike', u'expiat', u'paperwork', u'relay', u'relax', u'relat', u'notic', u'hurt', u'glass', u'exc', u'holi', u'midst', u'hold', u'accid', u'blade', u'conceptu', u'sweeter', u'etiquett', u'household', u'virtu', u'cautiou', u'caution', u'want', u'gargantuan', u'shuffl', u'unpaid', u'travel', u'classifi', u'hot', u'hop', u'turk', u'perspect', u'hoy', u'diagram', u'wrong', u'bea

In [134]:
def extract_features(dataset):       #features are bag of words. document is a list of words of a sentence 
    feature_vector=[]
    labels=[]
    for sentence,label in dataset:
        features = {}
        labels.append(label)
        for word in vocab:
            features[word] = 0
            if word in sentence:
                features[word] = 1
        feature_vector.append(features.values())        
    return {'feature':feature_vector,'labels':labels}        

In [139]:
features=extract_features(labeled_data)

In [140]:
labeled_data[0]

([u'made', u'lot', u'chang', u'career', u'also', u'life'], 2)

In [141]:
classifier=svm.SVC()

In [142]:
classifier.fit(features['feature'],features['labels'])
with open('svc_unigram_model','w') as f:
    pickle.dump(classifier,f)

In [11]:
sen='work life is bore'
classifier.classify(extract_features(sen.split()))

'neg'

In [21]:
dist = classifier.prob_classify(extract_features(sen.split()))
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))

neg: 0.585660
sli_pos: 0.035267
sli_neg: 0.230902
neutral: 0.123239
pos: 0.024931


In [22]:
polarity=0
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))
    polarity=max(polarity,dist.prob(label))
print polarity

neg: 0.585660
sli_pos: 0.035267
sli_neg: 0.230902
neutral: 0.123239
pos: 0.024931
0.585660041903
